<a href="https://colab.research.google.com/github/valentingracia/codingDojoDC/blob/main/salesPredictionProy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alumno : Valentin Gracia

se importa el dataset, y se despliega informacion basica

In [136]:
import pandas as pd
import numpy as np
sales_df=pd.read_csv("/content/drive/MyDrive/Coding Dojo/salesPrediction/sales_predictions.csv")
print(f"HEAD \n {sales_df.head(3)}")
print(f"INFO \n {sales_df.info()}")
#print(sales_df.describe())
#sales_df.dropna(inplace=True)
#print(sales_df.info())



HEAD 
   Item_Identifier  Item_Weight  ...        Outlet_Type  Item_Outlet_Sales
0           FDA15         9.30  ...  Supermarket Type1          3735.1380
1           DRC01         5.92  ...  Supermarket Type2           443.4228
2           FDN15        17.50  ...  Supermarket Type1          2097.2700

[3 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Ou

buscamos las entradas unicas de columna y correjimos entradas no normalizadas

In [137]:
sales_df.value_counts()

Item_Identifier  Item_Weight  Item_Fat_Content  Item_Visibility  Item_Type     Item_MRP  Outlet_Identifier  Outlet_Establishment_Year  Outlet_Size  Outlet_Location_Type  Outlet_Type        Item_Outlet_Sales
NCZ54            14.650       Low Fat           0.083699         Household     163.4552  OUT018             2009                       Medium       Tier 3                Supermarket Type2  2599.2832            1
FDJ04            18.000       Low Fat           0.124452         Frozen Foods  118.5124  OUT046             1997                       Small        Tier 1                Supermarket Type1  1303.6364            1
FDJ15            11.350       Regular           0.023322         Dairy         182.5608  OUT046             1997                       Small        Tier 1                Supermarket Type1  3307.6944            1
                                                0.023359         Dairy         184.4608  OUT049             1999                       Medium       Tier 1   

In [138]:
sales_df["Item_Fat_Content"].replace({"LF": "Low Fat", "reg": "Regular","low fat":"Low Fat"}, inplace=True)
sales_df["Item_Fat_Content"].value_counts()


Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

algunas vistas

In [139]:


sales_df.groupby(["Item_Fat_Content","Item_Type"])["Item_Outlet_Sales"].count()
(pd.pivot_table(sales_df, index=["Item_Type"], columns=["Item_Fat_Content"], values=["Item_Outlet_Sales"], aggfunc=np.sum,margins = True)).style.format('{0:,.0f}')

In [140]:
(pd.pivot_table(sales_df, index=["Outlet_Location_Type"], columns=["Item_Fat_Content"], values=["Item_Outlet_Sales"], aggfunc=np.sum,margins = True)).style.format('{0:,.0f}')

In [140]:
plt.plot()